In [80]:
import pandas as pd
import polars as pl
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import math

## Cargo los cursos de coursera
#### No hay nulos
#### Me quedo solo con la columna ['course_id']
#### No hay nulos en la tabla

In [34]:
df_c = pl.read_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\Coursera_courses.csv");

In [35]:
df_c = df_c.select(pl.col('course_id'))
pd_c = df_c.to_pandas()

In [ ]:
df_c.drop_nulls()

## Voy a usar la version pandas de la tabla para rabajar el titulo

In [38]:
pd_c[['course_id-split-0-lpxo', 
      'course_id-split-1-lpxo', 
      'course_id-split-2-lpxo', 
      'course_id-split-3-lpxo', 
      'course_id-split-4-lpxo', 
      'course_id-split-5-lpxo', 
      'course_id-split-6-lpxo', 
      'course_id-split-7-lpxo', 
      'course_id-split-8-lpxo']] = pd_c['course_id'].str.split('-', n=-1, expand=True, regex=None)

In [39]:
pd_c = pd_c[pd_c.columns[:1].tolist() + ['course_id-split-0-lpxo', 
                                         'course_id-split-1-lpxo', 
                                         'course_id-split-2-lpxo', 
                                         'course_id-split-3-lpxo', 
                                         'course_id-split-4-lpxo', 
                                         'course_id-split-5-lpxo', 
                                         'course_id-split-6-lpxo', 
                                         'course_id-split-7-lpxo', 
                                         'course_id-split-8-lpxo'] + pd_c.columns[1:-9].tolist()]

In [40]:
# Deleted columns course_id
pd_c.drop(['course_id'], axis=1, inplace=True)

In [41]:
pd_c.to_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics\wordcloud.csv")

In [42]:
pl_wordcloud = pl.read_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics\wordcloud.csv")

In [43]:
pl_wordcloud_final = pl.concat([pl_wordcloud.select(pl.col('course_id-split-0-lpxo').alias('course_id')),
                                pl_wordcloud.select(pl.col('course_id-split-1-lpxo').alias('course_id')), 
                                pl_wordcloud.select(pl.col('course_id-split-2-lpxo').alias('course_id')), 
                                pl_wordcloud.select(pl.col('course_id-split-3-lpxo').alias('course_id')), 
                                pl_wordcloud.select(pl.col('course_id-split-4-lpxo').alias('course_id')), 
                                pl_wordcloud.select(pl.col('course_id-split-5-lpxo').alias('course_id')), 
                                pl_wordcloud.select(pl.col('course_id-split-6-lpxo').alias('course_id')), 
                                pl_wordcloud.select(pl.col('course_id-split-7-lpxo').alias('course_id')), 
                                pl_wordcloud.select(pl.col('course_id-split-8-lpxo').alias('course_id'))],how='vertical')

In [ ]:
pl_wordcloud_final.drop_nulls()

### Salvo coursera

In [45]:
df_c.write_parquet(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics\coursera.parquet")
df_c.write_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\coursera.csv")
pl_wordcloud_final.write_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics\wordcloud.csv")

## Cargo los reviews a los cursos de coursera

In [64]:
df_r = pl.read_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\Coursera_reviews.csv")
pd_r = df_r.to_pandas()

In [65]:
df_r.null_count()

,reviews,reviewers,date_reviews,rating,course_id
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


In [66]:
pd_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0                 1454711 non-null  int64 
 1   reviews       1454711 non-null  object
 2   reviewers     1454711 non-null  object
 3   date_reviews  1454711 non-null  object
 4   rating        1454711 non-null  int64 
 5   course_id     1454711 non-null  object
dtypes: int64(2), object(4)
memory usage: 66.6+ MB


#### La tabla tiene 55.5 MB
#### Hago cambios en los tipos de datos para mejorar

In [67]:
pd_r['reviewers'] = pd_r['reviewers'].astype('category')

In [68]:
pd_r['rating'] = pd_r['rating'].astype('int16')

In [69]:
pd_r['course_id'] = pd_r['course_id'].astype('category')

In [70]:
pd_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype   
---  ------        --------------    -----   
 0                 1454711 non-null  int64   
 1   reviews       1454711 non-null  object  
 2   reviewers     1454711 non-null  category
 3   date_reviews  1454711 non-null  object  
 4   rating        1454711 non-null  int16   
 5   course_id     1454711 non-null  category
dtypes: category(2), int16(1), int64(1), object(2)
memory usage: 54.7+ MB


#### Termino rebajandola a 42.2 MB
#### Salvo

In [73]:
pd_r.to_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\Coursera_reviews.csv")
pd_r.to_parquet(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics\coursera.parquet")

In [75]:
df_r = pl.read_parquet(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics\coursera.parquet")

In [78]:
sr_r_id = pd_r.groupby("course_id")["rating"].count()
pd_r_id = pd.DataFrame(sr_r_id)
pd_r_id = pd_r_id.reset_index()
pd_r_id = pd_r_id.rename(columns={'rating':'cantidad'})

#### A partir de 
### pd_r_id
#### voy a obtener dos df llamados
### most_r
### lest_r
#### que son dos df ordenados de forma ascendente y descendente para tener a mano los id mas revisados y los id menos revisados

In [79]:
most_r = pd_r_id.sort_values(by='cantidad', ascending=False, na_position='last')
lest_r = pd_r_id.sort_values(by='cantidad', ascending=True, na_position='last')

## dfr_group_rating = (•	Nivel de ventas según rating)
### DataFrame con datos de Review Coursera agrupados por rating
### agrega(cantidad_cursos)

In [81]:
dfr_group_rating = (
  df_r
  .groupby('course_id')
  .agg(
  [
    pl.col('rating').mean().alias('rating_promedio'),
    pl.col('rating').count().alias('cantidad')  
  ]
  )  
)

In [82]:
dfr_group_rating = dfr_group_rating.with_column(pl.col('rating_promedio')
                   .apply(lambda x: math.modf(x)[1] if (math.modf(x)[0] > 0 and math.modf(x)[0] < 5) else round(x))                             
                   )

In [83]:
dfr_group_rating_prom = (
  dfr_group_rating
  .groupby('rating_promedio')
  .agg(
  [
    pl.col('course_id').count().alias('cantidad_cursos')
  ]
  )
  .sort('rating_promedio')  
)

In [84]:
dfr_group_rating_prom.to_pandas().hvplot('rating_promedio','cantidad_cursos')

:Curve   [rating_promedio]   (cantidad_cursos)

## Edx

In [133]:
df_e = pl.read_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\edx_courses.csv")
df_e = df_e.select(['title','n_enrolled','course_type','Level','subject','language','course_effort','course_length','price'])

In [134]:
df_e = df_e.drop_nulls()

In [135]:
pd_e = df_e.to_pandas()

### Convertir la columna course_effort mas la columna course_length en horas que serian
### las horas necesarias para terminar el curso

In [136]:
# Split course_effort on " "
pd_e[['course_effort-split-0-5j95', 
      'course_effort-split-1-5j95', 
      'course_effort-split-2-5j95', 
      'course_effort-split-3-5j95']] = pd_e['course_effort'].str.split(' ', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:8].tolist() + ['course_effort-split-0-5j95', 
                                         'course_effort-split-1-5j95', 
                                         'course_effort-split-2-5j95', 
                                         'course_effort-split-3-5j95'] + pd_e.columns[8:-4].tolist()]

In [137]:
# Split course_effort-split-0-5j95 on "-"
pd_e[['course_effort-split-0-5j95-split-0-671r']] = pd_e['course_effort-split-0-5j95'].str.split('-', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:11].tolist() + ['course_effort-split-0-5j95-split-0-671r'] + pd_e.columns[11:-1].tolist()]

In [138]:
# Split course_effort on " "
pd_e[['course_effort-split-0-exqx', 
      'course_effort-split-1-exqx', 
      'course_effort-split-2-exqx', 
      'course_effort-split-3-exqx']] = pd_e['course_effort'].str.split(' ', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:8].tolist() + ['course_effort-split-0-exqx', 
                                         'course_effort-split-1-exqx', 
                                         'course_effort-split-2-exqx', 
                                         'course_effort-split-3-exqx'] + pd_e.columns[8:-4].tolist()]

In [139]:
# Split course_effort-split-0-exqx on "-"
pd_e[['course_effort-split-0-exqx-split-0-jynp']] = pd_e['course_effort-split-0-exqx'].str.split('-', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:11].tolist() + ['course_effort-split-0-exqx-split-0-jynp'] + pd_e.columns[11:-1].tolist()]

In [140]:
# Split course_effort on " "
pd_e[['course_effort-split-0-jyiu', 
      'course_effort-split-1-jyiu', 
      'course_effort-split-2-jyiu', 
      'course_effort-split-3-jyiu']] = pd_e['course_effort'].str.split(' ', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:8].tolist() + ['course_effort-split-0-jyiu', 
                                         'course_effort-split-1-jyiu', 
                                         'course_effort-split-2-jyiu', 
                                         'course_effort-split-3-jyiu'] + pd_e.columns[8:-4].tolist()]

In [141]:
# Split course_effort-split-0-jyiu on "–"
pd_e[['course_effort-split-0-jyiu-split-0-o7o7', 
      'course_effort-split-0-jyiu-split-1-o7o7']] = pd_e['course_effort-split-0-jyiu'].str.split('–', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:11].tolist() + ['course_effort-split-0-jyiu-split-0-o7o7', 
                                          'course_effort-split-0-jyiu-split-1-o7o7'] + pd_e.columns[11:-2].tolist()]

In [142]:
# Deleted columns course_effort-split-0-jyiu-split-0-o7o7
pd_e.drop(['course_effort-split-0-jyiu-split-0-o7o7'], axis=1, inplace=True)

# Deleted columns course_effort-split-0-jyiu
pd_e.drop(['course_effort-split-0-jyiu'], axis=1, inplace=True)

# Deleted columns course_effort
pd_e.drop(['course_effort'], axis=1, inplace=True)

# Deleted columns course_effort-split-3-jyiu
pd_e.drop(['course_effort-split-3-jyiu'], axis=1, inplace=True)

# Deleted columns course_effort-split-2-jyiu
pd_e.drop(['course_effort-split-2-jyiu'], axis=1, inplace=True)

# Deleted columns course_effort-split-1-jyiu
pd_e.drop(['course_effort-split-1-jyiu'], axis=1, inplace=True)

In [143]:
# Split course_length on " "
pd_e[['course_length-split-0-3xns', 
      'course_length-split-1-3xns']] = pd_e['course_length'].str.split(' ', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:8].tolist() + ['course_length-split-0-3xns', 
                                         'course_length-split-1-3xns'] + pd_e.columns[8:-2].tolist()]

In [144]:
# Deleted columns course_length-split-1-3xns
pd_e.drop(['course_length-split-1-3xns'], axis=1, inplace=True)

# Deleted columns course_length
pd_e.drop(['course_length'], axis=1, inplace=True)

# Renamed columns horas
pd_e.rename(columns={'course_effort-split-0-jyiu-split-1-o7o7': 'horas'}, inplace=True)

# Renamed columns semanas
pd_e.rename(columns={'course_length-split-0-3xns': 'semanas'}, inplace=True)

pd_e.horas = pd_e.horas.astype('int16')
pd_e.semanas = pd_e.semanas.astype('int16')

pd_e['horas'] = pd_e.horas * pd_e.semanas

pd_e.drop(['semanas'], axis=1, inplace=True)

In [145]:
pd_e['horas']

0        6
1       28
2      216
3      195
4       28
      ... 
970     50
971     60
972     36
973      2
974     24
Name: horas, Length: 975, dtype: int16

In [146]:
pd_e.isnull().sum()

title                                      0
n_enrolled                                 0
course_type                                0
Level                                      0
subject                                    0
language                                   0
horas                                      0
course_effort-split-0-exqx                 0
course_effort-split-1-exqx                 0
course_effort-split-2-exqx                 0
course_effort-split-0-exqx-split-0-jynp    0
course_effort-split-3-exqx                 0
course_effort-split-0-5j95                 0
course_effort-split-1-5j95                 0
course_effort-split-2-5j95                 0
course_effort-split-0-5j95-split-0-671r    0
course_effort-split-3-5j95                 0
price                                      0
dtype: int64

## Extraer el precio dentro de la cadena de texto de la columna price 
## y transformar esta columna en numerico entero

In [147]:
# Split price on "$"
pd_e[['price-split-0-rf65', 
      'price-split-1-rf65']] = pd_e['price'].str.split('$', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:9].tolist() + ['price-split-0-rf65', 
                                         'price-split-1-rf65'] + pd_e.columns[9:-2].tolist()]

In [148]:
# Split price-split-1-rf65 on " "
pd_e[['price-split-1-rf65-split-0-6uec', 
      'price-split-1-rf65-split-1-6uec']] = pd_e['price-split-1-rf65'].str.split(' ', n=-1, expand=True, regex=None)
pd_e = pd_e[pd_e.columns[:11].tolist() + ['price-split-1-rf65-split-0-6uec', 
                                          'price-split-1-rf65-split-1-6uec'] + pd_e.columns[11:-2].tolist()]

In [149]:
# Deleted columns price-split-1-rf65-split-1-6uec
pd_e.drop(['price-split-1-rf65-split-1-6uec'], axis=1, inplace=True)

# Deleted columns price-split-1-rf65
pd_e.drop(['price-split-1-rf65'], axis=1, inplace=True)

# Deleted columns price-split-0-rf65
pd_e.drop(['price-split-0-rf65'], axis=1, inplace=True)

# Deleted columns price
pd_e.drop(['price'], axis=1, inplace=True)

# Renamed columns price
pd_e.rename(columns={'price-split-1-rf65-split-0-6uec': 'price'}, inplace=True)

# Changed price to dtype int
pd_e['price'] = pd_e['price'].astype('float16')

In [150]:
pd_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   title                                    975 non-null    object 
 1   n_enrolled                               975 non-null    object 
 2   course_type                              975 non-null    object 
 3   Level                                    975 non-null    object 
 4   subject                                  975 non-null    object 
 5   language                                 975 non-null    object 
 6   horas                                    975 non-null    int16  
 7   course_effort-split-0-exqx               975 non-null    object 
 8   course_effort-split-1-exqx               975 non-null    object 
 9   price                                    975 non-null    float16
 10  course_effort-split-2-exqx               975 non-n

In [152]:
pd_e.to_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\edx.csv")

In [156]:
df_e = pl.read_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\edx.csv")

In [157]:
df_e.write_parquet(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics\edx.parquet")

## Udemy

In [180]:
dtypes={'price':pl.Float64}
df_u = pl.read_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\udemy_courses.csv",dtypes=dtypes)

In [181]:
df_u = df_u.select(['course_title', 'price', 'num_subscribers', 'num_reviews', 'num_lectures', 'level', 'content_duration', 'published_timestamp', 'subject'])

In [182]:
pd_u = df_u.to_pandas()

In [183]:
pd_u.isnull().sum()

course_title           0
price                  0
num_subscribers        0
num_reviews            0
num_lectures           0
level                  0
content_duration       0
published_timestamp    0
subject                0
dtype: int64

In [184]:
pd_u.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   course_title         3678 non-null   object 
 1   price                3678 non-null   float64
 2   num_subscribers      3678 non-null   int64  
 3   num_reviews          3678 non-null   int64  
 4   num_lectures         3678 non-null   int64  
 5   level                3678 non-null   object 
 6   content_duration     3678 non-null   float64
 7   published_timestamp  3678 non-null   object 
 8   subject              3678 non-null   object 
dtypes: float64(2), int64(3), object(4)
memory usage: 258.7+ KB


In [185]:
df_u.write_parquet(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics\udemy.parquet")
df_u.write_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\udemy.csv")

## dfu_group_level (•	Nivel de ventas según nivel)
### DataFrame con datos de Udemy agrupado por nivel de complejidad
### Agrega(precio promedio, promedio de reseñas, promedio de cuantas clases tienen los cursos, promedio de duracion de los cursos) 

In [167]:
dfu_group_level = (
  df_u
  .groupby('level')
  .agg(
  [
    pl.col('price').mean().alias('precio_promedio'),
    pl.col('num_reviews').mean().alias('promedio_reseñas'),
    pl.col('num_lectures').mean().alias('cant_clases_promedio'),
    pl.col('content_duration').mean().alias('duracion_promedio(h)')
  ]
  )  
)

In [168]:
dfu_group_level

level,precio_promedio,promedio_reseñas,cant_clases_promedio,duracion_promedio(h)
str,f64,f64,f64,f64
"""All Levels""",73.022291,209.588906,45.981337,4.869656
"""Expert Level""",91.12069,40.224138,30.775862,2.90546
"""Intermediate L...",61.92399,92.674584,35.935867,3.733333
"""Beginner Level...",55.681102,101.633858,32.998425,3.091194


## dfu_group_level_subject (•	Nivel de ventas según nivel) + Tipo Cursos
### DataFrame con datos de Udemy agrupado por nivel de complejidad/tipo de cursos
### Agrega(precio promedio, promedio de reseñas, promedio de cuantas clases tienen los cursos, promedio de duracion de los cursos) 

In [169]:
dfu_group_level_subject = (
  df_u
  .groupby(['level','subject'])
  .agg(
  [
    pl.col('price').mean().alias('precio_promedio'),
    pl.col('num_reviews').mean().alias('promedio_reseñas'),
    pl.col('num_lectures').mean().alias('cant_clases_promedio'),
    pl.col('content_duration').mean().alias('duracion_promedio(h)')
  ]
  )  
)

## dfu_group_year
### Dataframe con datos de Udemy agrupado por años
### Agrega(Cantidad de cursos por año)

In [172]:
dfu_group_year = (
  pl.read_csv(r"C:\Users\ornal\Desktop\Data Analitics\PI03-Analytics-Origen\udemy.csv", parse_dates=True)
  .with_column(pl.col('published_timestamp').cast(pl.Datetime).dt.year().alias('year'))  
  .groupby('year')
  .agg(
  [
    pl.col('price').count().alias('cantidad_cursos')
  ]
  )
  .sort('year')  
)

In [173]:
dfu_group_year.to_pandas().hvplot('year','cantidad_cursos')

:Curve   [year]   (cantidad_cursos)

## dfu_group_precio (•	Nivel de ventas según precio.)
### DataFrame con datos de Udemy agrupados por precio
### Agrega(Cantidad Vendida)

In [174]:
dfu_group_precio = (
  df_u
  .filter(pl.col('price') != 0)  
  .groupby('price')
  .agg(
  [
    pl.col('price').count().alias('cantidad_cursos')
  ]
  )
  .sort('price')  
)

In [175]:
dfu_group_precio.to_pandas().hvplot('price','cantidad_cursos')

:Curve   [price]   (cantidad_cursos)

In [177]:
dfu_group_level1 = (
  df_u
  .groupby('level')
  .agg(
  [
    pl.col('price').mean().alias('precio_promedio'),
    pl.col('price').count().alias('cantidad_cursos')
  ]
  )
  .sort(['level','precio_promedio'])  
)
dfu_group_level1 = dfu_group_level1.with_column(pl.col('level').alias('nivel_complejidad'))
dfu_group_level1 = dfu_group_level1.select(pl.exclude('level'))

### Ordenar la columna level

In [178]:
dfu_group_level1 = (
       dfu_group_level1.with_column(pl.col('nivel_complejidad')
       .apply(lambda x: '1'+x if x == 'All Levels' else x)                 
       .apply(lambda x: '2'+x if x == 'Beginner Level' else x)                 
       .apply(lambda x: '3'+x if x == 'Intermediate Level' else x)                                  
       .apply(lambda x: '4'+x if x == 'Expert Level' else x)                                  
       .alias('level1'))
       .sort('level1') 
       ) 
dfu_group_level1 = dfu_group_level1.select(pl.exclude('level1'))
#df_u.head()

In [179]:
dfu_group_level1.to_pandas().hvplot('nivel_complejidad','cantidad_cursos')

:Curve   [nivel_complejidad]   (cantidad_cursos)

## Reviews